## [Spark SQL, DataFrames and  DataSets](https://spark.apache.org/docs/1.6.1/sql-programming-guide.html#spark-sql-dataframes-and-datasets-guide)
When working with datasets that are disk resident, there is a big advantage to using methods developed for Relational Database Management Systems (RDBMS). The corresponding API is called `pyspark SQL`. For us the main object of interest will be a single database table, which corresponds to the pyspark SQL `DataFrame`

## Dataframes 
Dataframes are a special type of RDDs. They are similar to, but not the same as, pandas dataframes. They are used to store two dimensional data, similar to the type of data stored in a spreadsheet. Each column in a dataframe can have a different type and each row contains a `record`.

Spark DataFrames are similar to `pandas` DataFrames. With the important difference that spark DataFrames are **distributed** data structures, based on RDDs.

In [1]:
from pyspark import SparkContext
sc = SparkContext(master="local[4]")

In [2]:
import os
import sys

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType

In [3]:
# Just like using Spark requires having a SparkContext, using SQL requires an SQLContext
sqlContext = SQLContext(sc)
sqlContext

### Constructing a DataFrame from an RDD of Rows
Each Row defines it's own  fields, the schema is *inferred*.

In [4]:
# One way to create a DataFrame is to first define an RDD from a list of rows
some_rdd = sc.parallelize([Row(name=u"John", age=19),
                           Row(name=u"Smith", age=23),
                           Row(name=u"Sarah", age=18)])
some_rdd.collect()

[Row(age=19, name=u'John'),
 Row(age=23, name=u'Smith'),
 Row(age=18, name=u'Sarah')]

In [5]:
# The DataFrame is created from the RDD or Rows
# Infer schema from the first row, create a DataFrame and print the schema
some_df = sqlContext.createDataFrame(some_rdd)
some_df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [6]:
# A dataframe is an RDD of rows plus information on the schema.
# performing **collect()* on either the RDD or the DataFrame gives the same result.
print type(some_rdd),type(some_df)
print 'some_df =',some_df.collect()
print 'some_rdd=',some_rdd.collect()

<class 'pyspark.rdd.RDD'> <class 'pyspark.sql.dataframe.DataFrame'>
some_df = [Row(age=19, name=u'John'), Row(age=23, name=u'Smith'), Row(age=18, name=u'Sarah')]
some_rdd= [Row(age=19, name=u'John'), Row(age=23, name=u'Smith'), Row(age=18, name=u'Sarah')]


### Defining the Schema explicitly
The advantage of creating a DataFrame using a pre-defined schema allows the content of the RDD to be simple tuples, rather than rows.

In [7]:
# In this case we create the dataframe from an RDD of tuples (rather than Rows) and provide the schema explicitly
another_rdd = sc.parallelize([("John", 19), ("Smith", 23), ("Sarah", 18)])
# Schema with two fields - person_name and person_age
schema = StructType([StructField("person_name", StringType(), False),
                     StructField("person_age", IntegerType(), False)])

# Create a DataFrame by applying the schema to the RDD and print the schema
another_df = sqlContext.createDataFrame(another_rdd, schema)
another_df.printSchema()
# root
#  |-- age: binteger (nullable = true)
#  |-- name: string (nullable = true)

root
 |-- person_name: string (nullable = false)
 |-- person_age: integer (nullable = false)



## Loading DataFrames from disk
There are many maethods to load DataFrames from Disk. Here we will discuss three of these methods
1. JSON 
2. CSV
3. Parquet

In addition, there are API's for connecting Spark to an external database. We will not discuss this type of connection in this class.

### Loading dataframes from JSON files
[JSON](http://www.json.org/) is a very popular readable file format for storing structured data.
Among it's many uses are **twitter**, `javascript` communication packets, and many others. In fact this notebook file (with the extension `.ipynb` is in json format. JSON can also be used to store tabular data and can be easily loaded into a dataframe.

In [8]:
# when loading json files you can specify either a single file or a directory containing many json files.
path = "../../Data/people.json"
!cat $path 

{"name":"Michael"}
{"name":"Andy", "age":30}
{"name":"Justin", "age":19}


In [9]:
# Create a DataFrame from the file(s) pointed to by path
people = sqlContext.read.json(path)
print 'people is a',type(people)
# The inferred schema can be visualized using the printSchema() method.
print
people.printSchema()
# root
#  |-- age: IntegerType
#  |-- name: StringType

people is a <class 'pyspark.sql.dataframe.DataFrame'>

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



### Excercise: Loading csv files into dataframes

Spark 2.0 includes a facility for reading csv files. In this excercise you are to create similar functionality using your own code.

You are to write a class called `csv_reader` which has the following methods:

* `__init__(self,filepath):` recieves as input the path to a csv file. It throws an exeption `NoSuchFile` if the file does not exist.
* `Infer_Schema()` opens the file, reads the first 10 lines (or less if the file is shorter), and infers the schema. The first line of the csv file defines the column names. The following lines should have the same number of columns and all of the elements of the column should be of the same type. The only types allowd are `int`,`float`,`string`. The method infers the types of the columns, checks that they are consistent, and defines a dataframe schema of the form:
```python
schema = StructType([StructField("person_name", StringType(), False),
                     StructField("person_age", IntegerType(), False)])
```
If everything checks out, the method defines a `self.` variable that stores the schema and returns the schema as it's output. If an error is found an exception `BadCsvFormat` is raised.
* `read_DataFrame()`: reads the file, parses it and creates a dataframe using the inferred schema. If one of the lines beyond the first 10 (i.e. a line that was not read by `InferSchema`) is not parsed correctly, the line is not added to the Dataframe. Instead, it is added to an RDD called `bad_lines`.
The methods returns the dateFrame and the `bad_lines` RDD.

### Parquet files
[Parquet](http://parquet.apache.org/) is a columnar format that is supported by many other data processing systems. Spark SQL provides support for both reading and writing Parquet files that automatically preserves the schema of the original data. 

#### More about Parquet
Parquet is a column-based file format and uses disk-resident data structure that support efficient access to subsets of the records. As a result, loading a subset of the records from a Parquet file is much more efficient than loading the same subset of records from a csv or json file. In addition, parquet is compatible with HDFS which further accelerates record retrieval in a distributed system.

In [13]:
dir='../../Data'
parquet_file=dir+"/users.parquet"
!ls $dir

Moby-Dick.txt example.csv   people.json   users.parquet
Weather       old_data      table.csv


In [14]:
#load a Parquet file
df = sqlContext.read.load(parquet_file)
df.show()

+------+--------------+----------------+
|  name|favorite_color|favorite_numbers|
+------+--------------+----------------+
|Alyssa|          null|  [3, 9, 15, 20]|
|   Ben|           red|              []|
+------+--------------+----------------+



In [15]:
df2=df.select("name", "favorite_color")
df2.show()

+------+--------------+
|  name|favorite_color|
+------+--------------+
|Alyssa|          null|
|   Ben|           red|
+------+--------------+



In [16]:
df2.write.save(dir+"/namesAndFavColors.parquet")
!ls -ld $dir/*.parquet

drwxr-xr-x  10 yoavfreund  staff  340 Apr 16 22:23 ../../Data/namesAndFavColors.parquet
-rw-r--r--   1 yoavfreund  staff  615 Apr 16 22:21 ../../Data/users.parquet


A new interface object has been added in **Spark 2.0** called **SparkSession**. A spark session is initialized using a `builder`. For example
```python
spark = SparkSession.builder \
         .master("local") \
         .appName("Word Count") \
         .config("spark.some.config.option", "some-value") \
         .getOrCreate()
```

Using a SparkSession a Parquet file is read [as follows:](http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.parquet):
```python
df = spark.read.parquet('python/test_support/sql/parquet_partitioned')
```

### Using SQL queries on DataFrames
You can use [Hive `select` syntax](https://cwiki.apache.org/confluence/display/Hive/LanguageManual+Select) to select a subset of the rows in a dataframe.
Spark supports a [subset](https://spark.apache.org/docs/latest/sql-programming-guide.html#supported-hive-features) of the Hive SQL query language

To use sql on a dataframe you need to first `register` it as a `TempTable`.

In [17]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [18]:
# Register this DataFrame as a table.
people.registerTempTable("people")

# SQL statements can be run by using the sql methods provided by sqlContext
teenagers = sqlContext.sql("SELECT name FROM people WHERE age >= 13 AND age <= 19")
for each in teenagers.collect():
    print(each[0])

Justin


### A somewhat bigger example
This file contains 1/1000 of the original file, which contains about 9 million lines and whose size is about 7GB.

In [19]:
%cd ../../Data
import urllib
# If these commands don't work, you can download the files using your rowser
f=urllib.urlretrieve("https://drive.google.com/open?id=0B8IGBNGc5gVANWtlUElzd0JxV1E")
#!curl  'https://drive.google.com/open?id=0B8IGBNGc5gVANWtlUElzd0JxV1E' > test.tgz
!ls -lrt 
#!tar xzvf  weather.tgz

/Users/yoavfreund/projects/edX-Micro-Master-in-Data-Science/big-data-analytics-using-spark/Data
total 2496
-rw-r--r--   1 yoavfreund  staff       73 Mar  1 17:51 people.json
lrwxr-xr-x   1 yoavfreund  staff      101 Mar  2 14:03 old_data -> /Users/yoavfreund/academic.papers/Courses/BigDataAnalytics/BigData_spring2016/UCSD_BigData_2016/Data/
-rw-r--r--   1 yoavfreund  staff       43 Mar  2 14:04 example.csv
-rw-r--r--   1 yoavfreund  staff       75 Mar  2 14:06 table.csv
drwxr-xr-x  16 yoavfreund  staff      544 Mar 24 20:15 Weather
-rw-r--r--   1 yoavfreund  staff  1257260 Apr 10 21:33 Moby-Dick.txt
-rw-r--r--   1 yoavfreund  staff      615 Apr 16 22:21 users.parquet
drwxr-xr-x  10 yoavfreund  staff      340 Apr 16 22:23 namesAndFavColors.parquet


In [22]:
f

('/var/folders/80/c2kfvdvx5cx570r4vlzqgb840000gq/T/tmpVzIBVL',
 <httplib.HTTPMessage instance at 0x1099d2488>)

In [ ]:
parquet_file=dir+"/Weather_sampled.parquet"
df = sqlContext.read.load(parquet_file)
print 'count=',df.count()
print 'columns=',df.columns[:5]

#### Count the number of occurances of each measurement

In [ ]:
L=df.groupBy('measurement').count().collect()
D=[(e.measurement,e['count']) for e in L]
sorted(D,key=lambda x:x[1], reverse=True)[:6]

### Reading only select lines
Suppose we are only interested in snow measurements. We can apply an SQL query directly to the 
parquet files. As the data is organized in columnar structure, we can do the selection efficiently without loading the whole file to memory.

Here the file is small, but in real applications it can consist of hundreds of millions of records. In such cases loading the data first to memory and then filtering it is very wasteful.

In [ ]:
query='SELECT station,measurement,year FROM parquet.`%s` WHERE measurement="SNOW"'%parquet_file
print query
df2 = sqlContext.sql(query)
print df2.count(),df2.columns

For more on DataFrames see: 
* [API for the DataFrame class](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame)

For more on Spark SQL see:
* [API for the pyspark.sql module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module)